In [5]:
# Installing great_expectations
!pip3 install great_expectations

In [6]:
#importing necessary libraries
import great_expectations as gx
import pandas as pd

In [26]:
# Mounting drive to connect to the dataset
from google.colab import drive
drive.mount('/content/drive')

# Defining the file path
file_path = "/content/drive/My Drive/Colab Notebooks/DE/status_data.xlsx"

# Reading the Excel file
my_data = pd.read_excel(file_path)

# Displaying the first few rows
my_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,cap_id,last_trip_date,captain_block_status,segment_,car_reg_yr,vehicle_status,lifetime_trips,AHs_30D,AHs_60D,AHs_90D,...,vehicle_registration_expired,vehicle_registration_expiry_date,vehicle_registration_at_risk,license_front_missing,profile_pic_missing,car_reg_front_missing,car_reg_back_missing,No_Criminal_Record_missing,Drug_Test_missing,risk_state
0,1,2024-12-28,PERM_BLOCKED,Casual Captains,2025.0,safe,8,NaN,14.40,14.40,...,0,2025-11-11,vehicle_registration_safe,0,0,0,0,1,1,Risk
1,2,2025-01-24,ACTIVE,Casual Captains,2012.0,safe,1,0.86,0.86,0.86,...,0,2027-02-17,vehicle_registration_safe,0,0,0,0,1,1,Risk
2,3,2025-01-31,ACTIVE,Casual Captains,2006.0,safe,1354,1.84,6.04,9.60,...,0,2025-06-05,vehicle_registration_safe,0,0,0,0,0,1,Safe
3,4,2025-01-11,ACTIVE,Aspiring Full Time,2010.0,safe,98,NaN,85.69,85.69,...,0,2025-09-26,vehicle_registration_safe,0,0,0,0,1,1,Risk
4,5,2024-12-31,ACTIVE,Casual Captains,2019.0,safe,19,NaN,12.68,16.52,...,0,2027-10-02,vehicle_registration_safe,0,0,0,0,1,1,Risk


# Using Great Expectations

In [27]:
# Setting context
context = gx.get_context()

# Setting data source
data_source = context.data_sources.add_pandas(name = "status")

# Setting up data asset
data_asset = data_source.add_dataframe_asset(name = "my_data")

batch_definition_name = "status_dataset"

batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)
assert batch_definition.name == batch_definition_name

# Define the Batch Parameters
batch_parameters = {"dataframe": my_data}
# Retrieve the Batch
batch = batch_definition.get_batch(batch_parameters=batch_parameters)


# Defining Expectations
expectation_suite_name = "status_suite"
suite = gx.ExpectationSuite(name=expectation_suite_name)

# Add Expectations to Suite
# 1.Adding expectation that "cap_id" must not be null
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="cap_id")
)

# 2.Adding expectation that "cap_id" must be unique
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeUnique(column="cap_id")
)

# 3.Adding expectation that available_hours in 30 days should be less than available_hours in 60 days
suite.add_expectation(
    gx.expectations.ExpectColumnPairValuesAToBeGreaterThanB(
        column_A="AHs_60D",
        column_B="AHs_30D"
    )
)

# 4.Adding expectation that available_hours in 60 days should be less than available_hours in 90 days
suite.add_expectation(
    gx.expectations.ExpectColumnPairValuesAToBeGreaterThanB(
        column_A="AHs_90D",
        column_B="AHs_60D"
    )
)

# 4.Adding expectation for age column to be greater than 18
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column = "age",
        min_value = 18
    )
)

# 5. Adding expectation that average age is greater than 18
suite.add_expectation(
    gx.expectations.ExpectColumnMeanToBeBetween(
        column = "age",
        min_value = 18
    )
)


# 6. Adding expectation that lifetime_trips cannot be negative
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column = "lifetime_trips",
        min_value = 0
    )
)

# Add the Expectation Suite to the Context
context.suites.add(suite)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmpqzbzy8i7' for ephemeral docs site


{
  "name": "status_suite",
  "id": "39ab7bd5-ef25-4a42-a505-5bcdadb4dddf",
  "expectations": [
    {
      "type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "cap_id"
      },
      "meta": {},
      "id": "1db3ca39-4b96-4149-a134-815652573bfc"
    },
    {
      "type": "expect_column_values_to_be_unique",
      "kwargs": {
        "column": "cap_id"
      },
      "meta": {},
      "id": "85efa0db-b87c-479d-8fa5-b5f10bf22eb1"
    },
    {
      "type": "expect_column_pair_values_a_to_be_greater_than_b",
      "kwargs": {
        "column_A": "AHs_60D",
        "column_B": "AHs_30D"
      },
      "meta": {},
      "id": "aedf7db6-7190-4355-8c21-26db3bfa23de"
    },
    {
      "type": "expect_column_pair_values_a_to_be_greater_than_b",
      "kwargs": {
        "column_A": "AHs_90D",
        "column_B": "AHs_60D"
      },
      "meta": {},
      "id": "2b99b934-7e89-495b-89d5-d4a6ed6470c7"
    },
    {
      "type": "expect_column_values_to_be_between"

In [28]:
# Validate the Data Against the Suite
validation_results = batch.validate(suite)

# Evaluate the Results
print(validation_results)

Calculating Metrics:   0%|          | 0/40 [00:00<?, ?it/s]

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "status-my_data",
          "column": "cap_id"
        },
        "meta": {},
        "id": "1db3ca39-4b96-4149-a134-815652573bfc"
      },
      "result": {
        "element_count": 31510,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": [],
        "partial_unexpected_index_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_unique",
        "kwargs": {
          "batch_id": "status-my_data",
          "column": "cap_id"
        },
        "meta": {},
        "id": "85efa0